# Pubmed Example

## Data

In [1]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from dgl.data import PubmedGraphDataset
dataset = PubmedGraphDataset()
graph = dataset[0]

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


We can save the data file in only one file `pubmed.npz`.

In [2]:
# pubmed_feat
node_feats = sparse.csr_matrix(graph.ndata["feat"].numpy())
# pubmed_class
node_class = graph.ndata["label"].numpy()  # (19717,)
# pubmed_edge
edge = torch.stack(graph.edges()).numpy().T
# pubmed only has 1 single graph, there are some isolated nodes in the graph. 
# These isolated nodes are added as zero-vecs into the right position
node_list = np.ones((1, graph.num_nodes()))  # (1, 19717)
edge_list = np.ones((1, graph.num_edges()))  # (1, 88651)

data = {
    "node_feats": node_feats,
    "node_class": node_class,
    "edge": edge,
    "node_list": node_list,
    "edge_list": edge_list
}

In [3]:
from gli.utils import save_data
save_data("pubmed", **data)

Save all dense arrays to pubmed.npz, including ['node_class', 'edge', 'node_list', 'edge_list']
Save sparse matrix node_feats to pubmed_node_feats.sparse.npz


In [4]:
pubmed = np.load("pubmed.npz", allow_pickle=True)
pubmed.files

['node_class', 'edge', 'node_list', 'edge_list']

In [5]:
!du pubmed.npz -h

624	pubmed.npz
du: -h: No such file or directory


## Task

In [6]:
train_set = graph.ndata["train_mask"].nonzero().squeeze().numpy()
val_set = graph.ndata["val_mask"].nonzero().squeeze().numpy()
test_set = graph.ndata["test_mask"].nonzero().squeeze().numpy()

In [7]:
task_data = {
    "train": train_set,
    "val": val_set,
    "test": test_set
}
np.savez_compressed("pubmed_task.npz", **task_data)